In [8]:
from dotenv import load_dotenv
from google import genai
load_dotenv() 

# Creating the client
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

In [9]:
# Simple test
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make decisions or predictions.


In [14]:
# Zero-shot prompting: No example or demonstration to the LLM
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents="Classify the text into neutral, negative or positive: Text: The time off was ok. Sentiment:"
)
print(response.text)

**Positive** (though very mildly so, leaning towards acceptable/satisfactory rather than enthusiastic).

"Ok" indicates that it was acceptable and not negative, implying a basic level of satisfaction.


In [ ]:
# Few-shot Enable in-context learning providing demonstration in the
# prompt to steer the model to better performance
prompt = '''
    
'''
response = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=prompt
)
print(response.text)
